In [21]:
import cv2
import os
import matplotlib.pyplot as plt

image = cv2.imread('/home/charlie/projects/entropy/img/Raw_superficial/reactive/active/35_1172576-1_20170822_OCT_00002576.bmp_superficial.jpg')
#plt.imshow(image)
img_gabor = Gabor_process(image)
cv2.imwrite('/home/charlie/projects/entropy/img/test/gabor2.jpg', img_gabor)

True

<Figure size 640x480 with 0 Axes>

In [28]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

# Grayscale
def BGR2GRAY(img):
    # Grayscale
    gray = 0.2126 * img[..., 2] + 0.7152 * img[..., 1] + 0.0722 * img[..., 0]
    return gray

# Gabor
def Gabor_filter(K_size=111, Sigma=10, Gamma=1.2, Lambda=10, Psi=0, angle=0):
    # get half size
    d = K_size // 2

    # prepare kernel
    gabor = np.zeros((K_size, K_size), dtype=np.float32)

    # each value
    for y in range(K_size):
        for x in range(K_size):
            # distance from center
            px = x - d
            py = y - d

            # degree -> radian
            theta = angle / 180. * np.pi

            # get kernel x
            _x = np.cos(theta) * px + np.sin(theta) * py

            # get kernel y
            _y = -np.sin(theta) * px + np.cos(theta) * py

            # fill kernel
            gabor[y, x] = np.exp(-(_x**2 + Gamma**2 * _y**2) / (2 * Sigma**2)) * np.cos(2*np.pi*_x/Lambda + Psi)

    # kernel normalization
    gabor /= np.sum(np.abs(gabor))

    return gabor


def Gabor_filtering(gray, K_size=111, Sigma=10, Gamma=1.2, Lambda=10, Psi=0, angle=0):
    # get shape
    H, W = gray.shape

    # padding
    gray = np.pad(gray, (K_size//2, K_size//2), 'edge')

    # prepare out image
    out = np.zeros((H, W), dtype=np.float32)

    # get gabor filter
    gabor = Gabor_filter(K_size=K_size, Sigma=Sigma, Gamma=Gamma, Lambda=Lambda, Psi=0, angle=angle)

    # filtering
    for y in range(H):
        for x in range(W):
            out[y, x] = np.sum(gray[y : y + K_size, x : x + K_size] * gabor)

    out = np.clip(out, 0, 255)
    out = out.astype(np.uint8)

    return out

def Gabor_process(img):
    # get shape
    H, W, _ = img.shape

    # gray scale
    gray = BGR2GRAY(img).astype(np.float32)

    # define angle
    As = [0, 45, 90, 135]

    # prepare pyplot
    plt.subplots_adjust(left=0, right=1, top=1, bottom=0, hspace=0, wspace=0.2)

    out = np.zeros([H, W], dtype=np.float32)

    # each angle
    for i, A in enumerate(As):
        # gabor filtering
        _out = Gabor_filtering(gray, K_size=111, Sigma=1.5, Gamma=1.2, Lambda=4, angle=A)
        # add gabor filtered image
        out += _out

    # scale normalization
    out = out / out.max() * 255
    #out = out.astype(np.uint8)
    kernel_size =5
    out = cv2.GaussianBlur(out,(kernel_size, kernel_size), 0)
    out = cv2.cvtColor(out, cv2.COLOR_BGR2GRAY)
    out = cv2.adaptiveThreshold(out, 255, cv2.ADAPTIVE_THRESH_MEAN_C, 
                                cv2.THRESH_BINARY, kernel_size, 0)
    return out

img= cv2.imread('/home/charlie/projects/entropy/raw/Raw_superficial/reactive/active/113_1296950-5_20180122_OCT_00000944.bmp_superficial.jpg')

print(img.shape)

img_gabor = Gabor_process(img)
cv2.imwrite('/home/charlie/projects/entropy/img/test/gabor5_mean.jpg', img_gabor)

(741, 741, 3)


error: OpenCV(4.7.0) /io/opencv/modules/imgproc/src/color.simd_helpers.hpp:92: error: (-2:Unspecified error) in function 'cv::impl::{anonymous}::CvtHelper<VScn, VDcn, VDepth, sizePolicy>::CvtHelper(cv::InputArray, cv::OutputArray, int) [with VScn = cv::impl::{anonymous}::Set<3, 4>; VDcn = cv::impl::{anonymous}::Set<1>; VDepth = cv::impl::{anonymous}::Set<0, 2, 5>; cv::impl::{anonymous}::SizePolicy sizePolicy = cv::impl::<unnamed>::NONE; cv::InputArray = const cv::_InputArray&; cv::OutputArray = const cv::_OutputArray&]'
> Invalid number of channels in input image:
>     'VScn::contains(scn)'
> where
>     'scn' is 1


<Figure size 640x480 with 0 Axes>

In [35]:
img_threshold = cv2.imread('/home/charlie/projects/entropy/raw/Raw_superficial/reactive/active/110_1296950-5_20170919_OCT_00000949.bmp_superficial.jpg')
img_threshold = cv2.cvtColor(img_threshold, cv2.COLOR_BGR2GRAY)
_, img_threshold = cv2.threshold(img_threshold,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)

cv2.imwrite('/home/charlie/projects/entropy/img/test/otsu.jpg', img_threshold)
                           

True

In [4]:
#moments
import cv2
import numpy as np

data = cv2.imread('/home/charlie/projects/entropy/raw/Raw_superficial/reactive/active/110_1296950-5_20170919_OCT_00000949.bmp_superficial.jpg')
total = 0
m0 = 1.0
m1 = 0.0
m2 = 0.0
m3 = 0.0
sum = 0.0
p0 = 0.0

histo = np.zeros(data.shape)
print(histo)

for i in range(data.shape[0]):
    total += data[i]

for i in range(data.shape[0]):
    histo[i] = data[i] / total

for i in range(data.shape[0]):
    m1 += i * histo[i]
    m2 += i * i * histo[i]
    m3 += i * i * i * histo[i]

cd = m0 * m2 - m1 * m1
c0 = (-m2 * m2 + m1 * m3) / cd
c1 = (m0 * -m3 + m2 * m1) / cd
z0 = 0.5 * (-c1 - np.sqrt(c1 * c1 - 4.0 * c0))
z1 = 0.5 * (-c1 + np.sqrt(c1 * c1 - 4.0 * c0))
p0 = (z1 - m1) / (z1 - z0)

summation = 0
threshold = -1
for i in range(data.shape[0]):
    print(histo[i])
    summation += histo[i]
    if summation > p0:
        threshold = i
        break

print(threshold)

[[[0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  ...
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]]

 [[0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  ...
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]]

 [[0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  ...
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]]

 ...

 [[0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  ...
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]]

 [[0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  ...
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]]

 [[0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  ...
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]]]
[[1.26020408 1.26666667 0.67741935]
 [6.         1.41176471 0.94805195]
 [2.26415094 1.04366812 0.63095238]
 ...
 [1.26737968 2.10619469 1.84444444]
 [1.2029703  1.07964602 0.74178404]
 [1.45238095 1.34065934 0.60986547]]


/tmp/ipykernel_288076/2922836885.py:21: RuntimeWarning: divide by zero encountered in true_divide
  histo[i] = data[i] / total
/tmp/ipykernel_288076/2922836885.py:21: RuntimeWarning: invalid value encountered in true_divide
  histo[i] = data[i] / total
/tmp/ipykernel_288076/2922836885.py:24: RuntimeWarning: invalid value encountered in multiply
  m1 += i * histo[i]
/tmp/ipykernel_288076/2922836885.py:25: RuntimeWarning: invalid value encountered in multiply
  m2 += i * i * histo[i]
/tmp/ipykernel_288076/2922836885.py:26: RuntimeWarning: invalid value encountered in multiply
  m3 += i * i * i * histo[i]


ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [10]:
#moments
import cv2
import numpy as np

def partial_sum(y, j):
    x = 0
    for i in range(j + 1):
        x += y[i]
    return x

img = cv2.imread('/home/charlie/projects/entropy/raw/Raw_superficial/reactive/active/110_1296950-5_20170919_OCT_00000949.bmp_superficial.jpg')

data = img.flatten()
for i in range(len(data)):
    if data[i] > 0:
        maxbin = i

for i in range(len(data) - 1, -1, -1):
    if data[i] > 0:
        minbin = i

data2 = data[minbin:maxbin + 1]
total = data2.sum()
temp = float("inf")
threshold = 0
avec = [0] * 256
#print(avec)
for i in range(256):
    ptile = (i + 1) / 256.0
    avec[i] = abs((partial_sum(data2, i) / total) - ptile)
    print(avec[i])
    if avec[i] < temp:
        temp = avec[i]
        threshold = i
print(threshold)
img[img > threshold] = 255
img[img <= threshold] = 0


inf
0.003903776394125773
0.007807552788251546
0.011712330642245427
0.015616177138561968
0.01952002363487851
0.02342481150347107
0.02732865799978761
0.031232514510702834
0.03513717218951254
0.03904103871502644
0.04294490524054034
0.04684947278796192
0.05075343945946264
0.054657416145562035
0.05856200372218097
0.06246594034988564
0.06636985694839295
0.07027436440822245
0.07417830103592711
0.07808221763443442
0.08198672509426391
0.08589069176576462
0.08979461837887061
0.09369913585329878
0.09760309251020081
0.10150703915250417
0.10541154661233366
0.10931549325463702
0.11321942988234168
0.1171239273275725
0.12102785394067848
0.12493179056838316
0.12883630804281132
0.13274022464131863
0.1366441512544246
0.14054865871425412
0.14445257531276143
0.14835649191126873
0.1522610093856969
0.1561649359988029
0.16006887262650757
0.16397339010093573
0.1678773367432391
0.17178127337094376
0.17568577081617456
0.1795897274730766
0.18349367411537995
0.18739818157520943
0.19130210818831545
0.19520603480142

In [32]:
#0206 medianBlur test
import cv2
import numpy as np

img = cv2.imread('/home/charlie/projects/entropy/raw/Raw_superficial/reactive/active/6_1054482-8_20170926_OCT_00000256.bmp_superficial.jpg',cv2.IMREAD_GRAYSCALE)

blur_img = cv2.medianBlur(img,5) 

kernel_size=333

def do_mean(img):
    im2 = img.copy()
    im2 = cv2.adaptiveThreshold(im2, 255, cv2.ADAPTIVE_THRESH_MEAN_C, 
                                cv2.THRESH_BINARY, kernel_size, 2)
    return(im2)

def do_gaussmean(img):
    im2 = img.copy()
    im2 = cv2.adaptiveThreshold(im2, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, 
                                cv2.THRESH_BINARY, kernel_size, 2)
    return(im2)

def do_drawcontour(img):
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3,3))
    im2 = cv2.morphologyEx(img, cv2.MORPH_OPEN, kernel, iterations=1)
    cnts = cv2.findContours(im2, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cnts = cnts[0] if len(cnts) == 2 else cnts[1]

    for c in cnts:
        area = cv2.contourArea(c)
        if area < 150:
            cv2.drawContours(im2, [c], -1, (0,0,0), -1)
    
    return im2
        
mean = do_mean(blur_img)
mean2 = do_drawcontour(mean)
gaussmean = do_gaussmean(blur_img)
gaussmean2 = do_drawcontour(gaussmean)

cv2.imwrite('/home/charlie/projects/entropy/test/blur.jpg', blur_img)
cv2.imwrite('/home/charlie/projects/entropy/test/mean.jpg', mean)
cv2.imwrite('/home/charlie/projects/entropy/test/mean2.jpg', mean2)

True